TASK 4 (Beginner LEVEL TASK) Stock Market Prediction and Forecasting using Stacked LSTM

Objective: To build a model using Recurrent Neural Networks (RNN) and especially Long-Short Term Memory model (LSTM) to predict future stock market values.

Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Reading the dataset

In [ ]:
url= "https://raw.githubusercontent.com/mwitiderrick/stockprice/master/NSE-TATAGLOBAL.csv"
df=pd.read_csv(url)

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
print(df.shape)
print(len(df))

In [ ]:
df.info()
df.describe()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df.sort_values(by='Date',ignore_index=True,inplace=True) #Sorting Values w.r.t the dates
df

In [ ]:
df_close = df.reset_index()['Close']
df_close

Plotting the Close value

In [ ]:
df_close.plot()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scale = MinMaxScaler(feature_range=(0,1))
df1 = scale.fit_transform(np.array(df_close).reshape(-1,1))
df1

Spiltting the data into train and test

In [ ]:
train_size = int(len(df1)*0.80)
test_size = len(df1)-train_size
train_data = df1[0:train_size,:]
test_data = df1[train_size:len(df1),:]

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
time_step = 100
x_train,y_train = [],[]
for i in range(len(train_data)-time_step-1):
    a = train_data[i:(i+time_step),0]
    x_train.append(a)
    y_train.append(train_data[i+ time_step,0])
x_train,y_train = np.array(x_train),np.array(y_train)

In [ ]:
x_test,y_test = [],[]
for i in range(len(test_data)-time_step-1):
    b = test_data[i:(i+time_step),0]
    x_test.append(b)
    y_test.append(test_data[i+time_step,0])
x_test,y_test = np.array(x_test),np.array(y_test)

In [ ]:
print("X_train shape :", x_train.shape)
print("Y_train shape :", y_train.shape)
print("X_test shape  :", x_test.shape)
print("Y_test shape  :", y_test.shape)

In [ ]:
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],1)
x_test  = x_test.reshape(x_test.shape[0],x_test.shape[1],1)

In [ ]:
print("X_train shape :", x_train.shape)
print("Y_train shape :", y_train.shape)
print("X_test shape  :", x_test.shape)
print("Y_test shape  :", y_test.shape)

Importing libraries for the Neural Network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Dropout

Building the Model

In [ ]:
model = Sequential()
#Adding the 1st layer of stacked lstm model
model.add(LSTM(units=50, return_sequences=True,input_shape=(x_train.shape[1],x_train.shape[2])))
model.add(Dropout(0.2))

In [ ]:
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

In [ ]:
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

In [ ]:
model.add(LSTM(units=50))
model.add(Dropout(0.2))

In [ ]:
model.add(Dense(units=1))

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

In [ ]:
model.fit(x=x_train,  y=y_train, batch_size=32, epochs=100,   validation_data=(x_test,y_test), verbose=1)

Prediction

In [ ]:
train_pred = model.predict(x_train)
test_pred = model.predict(x_test)

In [ ]:
train_pred = scale.inverse_transform(train_pred)
test_pred = scale.inverse_transform(test_pred)

Evaluation

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
sqrt(mean_squared_error(y_train,train_pred))

In [ ]:
sqrt(mean_squared_error(y_test, test_pred))

In [ ]:
lookback = 100
train_pred_plot = np.empty_like(df1)
train_pred_plot[:,:] = np.nan
train_pred_plot[lookback: len(train_pred)+lookback, :] = train_pred

test_pred_plot = np.empty_like(df1)
test_pred_plot[:,:] = np.nan
test_pred_plot[len(train_pred)+(lookback*2)+1: len(df1)-1, : ] = test_pred

Plotting the Baselines and Predictions

In [ ]:
plt.plot(scale.inverse_transform(df1),label='baseline')
plt.plot(train_pred_plot,label='train_predictions')
plt.plot(test_pred_plot,label= 'test_predictions')
plt.legend()
plt.show()

In [ ]:
len(test_data)

In [ ]:
x_input = test_data[307:].reshape(1,-1)
x_input.shape

In [ ]:
temp_input=list(x_input)
temp_input=temp_input[0].tolist()
temp_input

Forecasting the Prediction upto 20 Days

In [ ]:
output=[]
n_steps=100
i=0
while(i<21):
    
    if(len(temp_input)>100):
        #print(temp_input)
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
        output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        output.extend(yhat.tolist())
        i=i+1
    

print(output)

In [ ]:
df = df1.tolist()
df.extend(output)
df=scale.inverse_transform(df).tolist()
plt.plot(df)